# What is Dask

* [Dask](https://dask.org/) is a free and open-source parallel computing library that scales the existing Python ecosystem.
* Dask helps you scale your data science and machine learning workflows. 
* Dask makes it easy to work with Numpy, pandas, and Scikit-Learn etc.
* Dask is a framework to build distributed applications.
* Dask can scale down to your laptop and up to a cluster. We will use today on an environment you can set up on your computer.


Dask can be split into **two components**:

* **Collections**:  

Dask provides high-level Array, Bag, and DataFrame collections that mimic NumPy, lists, and Pandas. The advantage is that in can run in parallel on data that cannot fit in memory.
* **Schedulers**:

Dask provides schedulers to run the tasks in parallel

## Examples
We will go over some concepts of Dask that we will need today.

### Dask Array

Dask arrays combine many [NumPy](https://numpy.org/) arrays, arranged into chunks within a grid

Create an array of numbers represented by several NumPy arrays of size 10x10 (the arrays will be smaller if they cannot be divided evenly).

In [66]:
import dask.array as da
x = da.random.random((100, 100), chunks=(10, 10))
x

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>

Use NumPy syntax for operation

In [67]:
y = x + x.T
z = y[::2, 50:].mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(50,), dtype=float64, chunksize=(5,), chunktype=numpy.ndarray>

To get the result as a NumPy array

In [68]:
z.compute()

array([1.00768384, 0.97355635, 1.02757662, 1.04160331, 0.91841035,
       1.04022224, 0.94372283, 1.04872293, 0.9603477 , 1.00460021,
       0.94901949, 0.99186142, 0.94390023, 0.9327565 , 0.95628512,
       0.9017809 , 0.945887  , 0.92952669, 0.95288276, 0.98573597,
       0.9804065 , 1.00161442, 0.95374218, 1.01826068, 0.90925307,
       1.02319107, 0.98381396, 1.0700391 , 1.02105228, 1.01288914,
       1.10818052, 1.0039941 , 1.17158242, 0.9454848 , 1.02162172,
       0.96442781, 0.99902991, 0.97174576, 1.10945926, 1.05895171,
       0.97838394, 1.09438346, 1.00091608, 1.06286117, 0.99734373,
       1.07101088, 1.03618268, 0.96587652, 1.02591546, 1.04075181])

Depending on the available RAM, you can persist data in memory to speed up further computation

In [ ]:
y = y.persist()

To find the time it takes to perform an operation 

In [4]:
%time y.sum().compute()

CPU times: user 65.9 ms, sys: 17 ms, total: 82.9 ms
Wall time: 81.3 ms


10070.477327750283

## Dask Delayed 

``for`` loops are a common part to parallelize e.g. iterate over all the 2D-planes of a Z-stack.

Below we show how to parallelize sequential incrementation of each value using ``dask.delayed``.

In [25]:
data = [1, 2, 3, 4, 5, 6, 7, 8]

In [26]:
from time import sleep

def increment(x):
    sleep(1)
    return x + 1

In [63]:
%%time
# Usual way of running
results = []
for x in data:
    y = increment(x)
    results.append(y)
    
total = sum(results)

print("Compute:", total) 

Compute: 44
CPU times: user 388 ms, sys: 110 ms, total: 497 ms
Wall time: 8.03 s


We will "transform" our function to use ``dask.delayed``. 
The code below will finish **very quickly**. It will record what we want to compute as a task into a graph that will run later on parallel hardware.

In [32]:
from dask import delayed

In [60]:
# No computation happens
results = []

for x in data:
    y = delayed(increment)(x)
    results.append(y)
    
total = delayed(sum)(results)
print("Before computing:", total) 

Before computing: Delayed('sum-6ecc2071-9136-4909-a60b-508c81782908')


To get the result, we need to invoke the ``compute`` method.

In [61]:
%time result = total.compute()
print("After computing :", result)  # After it's computed

CPU times: user 58.2 ms, sys: 20.4 ms, total: 78.7 ms
Wall time: 1.02 s
After computing : 44


There are a few tricks with ``dask.delayed``. Please check [dask.delayed best practises](https://docs.dask.org/en/latest/delayed-best-practices.html)

### Parallelize the code below using ``dask.delayed``

We create a 3D-numpy array mimicing a z-stack.
We want to segment the XY-plane using ``dask_image``.
Run the segmentation below in parallel.
Hint: the function should return the label_image and the z-section so we can identify it.

```
planes = da.random.random((10, 100, 100), chunks=(10, 10, 10))
print(planes.shape)
```

```
import dask_image.ndfilters
import dask_image.ndmeasure

for z in range(planes.shape[0]):
    plane = planes[z, :, :]
    smoothed_image = dask_image.ndfilters.gaussian_filter(plane, sigma=[1, 1])
    threshold_value = 0.33 * da.max(smoothed_image).compute()
    threshold_image = smoothed_image > threshold_value
    label_image, num_labels = dask_image.ndmeasure.label(threshold_image)
```    

### Dask cluster 

Your computer will have multiple cores e.g. 4. When writing regular Python code, you are probably only using 1 of them. If you are using Numpy, e.g. for matrix multiplication, you will be using multiple cores because Numpy knows how to it but general Python code doesn't.

Dask cluster allows you to use multiple cores on your computer.
Dask has also a dashboard that you can use to monitor your work.

This time we will use a local cluster to increment each value in the ``data`` array define above.

In [41]:
def prepare_call(client):
    futures = []
    for x in data:
        y = client.submit(increment, x)
        futures.append(y)
    return futures

Create a local cluster

In [42]:
from dask.distributed import Client, LocalCluster

In [43]:
# if you want to specify number of workers etc.
cluster = LocalCluster(n_workers=2, processes=True, threads_per_worker=1)
# or simply 
# custer = LocalCluster()
with Client(cluster) as client:
    # perform code
    futures = prepare_call(client)
    results = client.gather(futures)

print(results)

[2, 3, 4, 5, 6, 7, 8, 9]


## Parallelize using LocalCluster

Using the 3D data above, parallelize the code this time using a LocalCluster instead of ``dask.delayed``.

### License (BSD 2-Clause)
Copyright (C) 2022 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.